In [1]:
import numpy as np

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


INSPIRATION FROM THE FOLLOWING BLOG 

https://medium.com/aimonks/traveling-salesman-problem-tsp-using-genetic-algorithm-fea640713758

In [2]:
from random import randrange, random

In [3]:
# set up country list

cities = [
    "Paris", "Berlin", "London", "Madrid", "Rome", "Amsterdam", "Lisbon", "Prague", "Vienna", "Stockholm",
]

len(cities)

10

In [4]:
def determine_coords(city_list, max_distance):
    country_coords = {}
    for city in city_list:
        while len(country_coords) < len(city_list):
            coords = (randrange(0, max_distance), randrange(0, max_distance))
            if coords not in country_coords.values():
                country_coords[city] = coords
                break

    return country_coords

coords = determine_coords(cities, 20)
print(len(coords))

coords

10


{'Paris': (14, 16),
 'Berlin': (7, 9),
 'London': (10, 9),
 'Madrid': (16, 17),
 'Rome': (6, 16),
 'Amsterdam': (6, 17),
 'Lisbon': (6, 7),
 'Prague': (1, 19),
 'Vienna': (11, 4),
 'Stockholm': (3, 9)}

In [6]:
# out of above countries, select N countries to construct initial population

def initial_countries(n_sol, n_countries, country_list):
    selected_cities = []  # list of n countries to permute
    while len(selected_cities) < n_sol:
        sublist_cities = []  # list of n countries to permute
        while len(sublist_cities) < n_countries:
            rand = randrange(0, len(country_list))
            selected_city = country_list[rand]
            if selected_city not in sublist_cities:
                sublist_cities.append(selected_city)
        if len(set(sublist_cities)) == n_countries:  # Ensure unique cities
            selected_cities.append(sublist_cities)

    return selected_cities

cities_selection = initial_countries(100, 10, cities)

In [10]:
def construct_pop_coords(solution_arr, coord_map):
    result_dict_list = []

    for subarray in solution_arr:
        subarray_dict = {city: coord_map.get(city, None) for city in subarray}
        result_dict_list.append(subarray_dict)

    return result_dict_list

solutions_coords = construct_pop_coords(cities_selection, coords)

solutions_coords[0]

{'Stockholm': (3, 9),
 'Rome': (6, 16),
 'Vienna': (11, 4),
 'Amsterdam': (6, 17),
 'Berlin': (7, 9),
 'Paris': (14, 16),
 'Madrid': (16, 17),
 'Lisbon': (6, 7),
 'Prague': (1, 19),
 'London': (10, 9)}

In [22]:
from math import dist

In [35]:
def calc_total_dist(sol_coords):
    max_range = len(sol_coords[0]) - 1
    total_dist_per_sol = []
    for sol_coord in sol_coords:
        solution_total_dist = 0
        for i in range(max_range):
            city_1_coords = list(sol_coord.values())[i]
            city_2_coords = list(sol_coord.values())[i + 1]
            
            distance = dist(city_1_coords, city_2_coords)
            solution_total_dist += distance 
            
        total_dist_per_sol.append(solution_total_dist)
    return total_dist_per_sol
        
total_distances_per_sol = calc_total_dist(solutions_coords)

In [40]:
def assign_weights(distance_sum_list):
    score_list = []
    for distance_sum in distance_sum_list:
        score_list.append(1 / (1 + distance_sum))

    return score_list

weights = assign_weights(total_distances_per_sol)

len(weights)

100

In [41]:
def probability_wheel(solutions, weights_list):
    total_score = sum(weights_list)
    probabilities = [x / total_score for x in weights_list] # gets proportion of each solution based on score (adds to 1)
    selection = []
    for i in range(len(solutions)):
        rand = random() # number between 0 and 1
        prob_sum = 0 # cumulative probability
        for idx, prob in enumerate(probabilities):
            prob_sum += prob # adds up to total
            if prob_sum > rand: # cutoff is exceeded
                selection.append(solutions[idx])
                break

    return selection

selected_solutions = probability_wheel(solutions_coords, weights)

In [42]:
def crossover(solutions, cross_rate): # TO DO : IMPLEMENT CROSSOVER RATE
    parents = [(solutions[i], solutions[i + 1]) for i in range(0, len(solutions), 2)]
    cross_solutions = [] # store the newly crossovered solutions
    for parent1, parent2 in parents:
        sub_cross_solutions_c1 = {}
        sub_cross_solutions_c2 = {}

        p1_city = [city for city in parent1.keys()]
        p2_city = [city for city in parent2.keys()]
        p1_coords = [coord for coord in parent1.values()]
        p2_coords = [coord for coord in parent2.values()]

        rand = randrange(1, len(solutions[0]))

        child_1_cities_cross = p1_city[:rand]
        child_2_cities_cross = p2_city[rand:]
        child_1_coord_cross = p1_coords[:rand]
        child_2_coord_cross = p2_coords[rand:]

        # this was in fact found in the blog mentioned in the sources above, i was just unaware the "+=" could be used on lists
        child_1_cities_cross += [city for city in child_2_cities_cross if city not in child_1_cities_cross]
        child_2_cities_cross += [city for city in child_1_cities_cross if city not in child_2_cities_cross]
        child_1_coord_cross += [coord for coord in child_2_coord_cross if coord not in child_1_coord_cross]
        child_2_coord_cross += [coord for coord in child_1_coord_cross if coord not in child_2_coord_cross]

        # this block of code reverts the crossover if there were any duplicates (code above was problematic)
        if child_1_cities_cross != len(p1_city):
            child_1_cities_cross = list(parent1.keys()) # if unable to get to length 10, kill crossover
            child_1_coord_cross = list(parent1.values())
        if child_2_cities_cross != len(p2_city):
            child_2_cities_cross = list(parent2.keys())
            child_2_coord_cross = list(parent2.values())

        for idx, city in enumerate(child_1_cities_cross):
            sub_cross_solutions_c1[city] = child_1_coord_cross[idx]
        for idx, city in enumerate(child_2_cities_cross):
            sub_cross_solutions_c2[city] = child_2_coord_cross[idx]

        cross_solutions.append(sub_cross_solutions_c1)
        cross_solutions.append(sub_cross_solutions_c2)

    return cross_solutions

cross_sample = crossover(selected_solutions, 0.5)

In [53]:
def mutate(city_coord_map, solutions, mutate_rate):
    mutated_solutions = []
    city_names = [city for city in city_coord_map.keys()]
    city_coords = [coords for coords in city_coord_map.values()]
    for solution in solutions:
        cities = [city for city in solution.keys()]
        coords = [coord for coord in solution.values()]
        for idx, current_city in enumerate(cities):
            rand = random()
            rand_idx = randrange(0, len(city_coord_map))
            new_city = city_names[rand_idx]
            new_coord = city_coords[rand_idx]
            if rand < mutate_rate and new_city not in cities:
                cities[idx] = new_city
                coords[idx] = new_coord

        mutated_solutions.append(dict(zip(cities, coords))) # maybe refactor this thing at some point if we want to

    return mutated_solutions

mutated_results = mutate(coords, cross_sample, 0.10)

In [58]:
def genetic_algorithm(cities, max_distance, n_sol, n_cities, n_iterations, cross_rate, mutate_rate):
    coords = determine_coords(cities, max_distance)
    cities_selection = initial_countries(n_sol, n_cities, cities)
    solutions_coords = construct_pop_coords(cities_selection, coords)
    
    optimal_solution = None
    optimal_dist = 10000
    optimal_score = 0
    
    for iteration in range(n_iterations):
        total_distances = calc_total_dist(solutions_coords)
        weights = assign_weights(total_distances)
        selected_solutions = probability_wheel(solutions_coords, weights)

        crossover_results = crossover(selected_solutions, cross_rate)

        mutated_results = mutate(coords, crossover_results, mutate_rate)

        total_distances_after_algo = calc_total_dist(mutated_results)
        weights_new_gen = assign_weights(total_distances_after_algo)

        best_dist = total_distances_after_algo[np.argmax(weights_new_gen)]
        best_sol = mutated_results[np.argmax(weights_new_gen)]

        if best_dist < optimal_dist:
            optimal_dist = best_dist
            optimal_solution = best_sol

        solutions_coords = mutated_results

    print(optimal_solution)
    print(optimal_dist)

genetic_algorithm(cities = cities, max_distance = 20, n_sol = 100, n_cities = 10, n_iterations = 10000, cross_rate = 0.4, mutate_rate = 0.2)

{'Stockholm': (16, 14), 'Amsterdam': (9, 18), 'Rome': (9, 17), 'Madrid': (8, 14), 'Paris': (10, 12), 'Vienna': (10, 11), 'London': (10, 10), 'Berlin': (9, 6), 'Prague': (19, 13), 'Lisbon': (16, 15)}
36.98817505002847
